In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)

In [ ]:
def softplus(input):
    return np.log(1 + np.exp(input))

def softplus_der(input):
    return np.exp(input) / (1 + np.exp(input))

In [ ]:
class FFNet: 
    def __init__(self):
        # internal naming vs youtube naming:
        # self.w1 = [[w1], [w2]]
        # self.b1 = [[b1], [b2]]
        # self.w2 = [[w3], [w4]]
        # self.b2 = [[b3]]
        
        self.x = np.array([0., 0.5, 1.])
        
        # target:       self.w1 = np.array([[3.34], [-3.53]])
        # youtube init: self.w1 = np.array([[2.74], [-1.13]])
        self.w1 = np.random.normal(size=(2, 1))
        print(f'{self.w1=}')

        # target:     self.b1 = np.array([[-1.43], [0.57]])
        self.b1 = np.zeros(shape=(2, 1))

        # target:       self.w2 = np.array([[-1.22], [-2.3]])
        # youtube init: self.w2 = np.array([[0.36], [0.63]])
        self.w2 = np.random.normal(size=(2, 1))
        print(f'{self.w2=}')
        
        self.y = np.array([0., 1., 0.])
        self.b2 = 0.
        self.step_size = 0.01

    def __repr__(self):
        return f'{self.w1 = }\n{self.b1 = }\n{self.w2 = }\n{self.b2 = }'
    
    def calc(self, x):
        self.a1 = x * self.w1 + self.b1
        self.z1 = softplus(self.a1)
        self.a2 = self.z1 * self.w2
        self.z2 = np.sum(self.a2, axis=0) + self.b2
        return self.z2

    def backpropagate(self, it):
        print(f'Iter: {it}')
        y_dash = self.calc(self.x)
        res = self.y - y_dash
        res_sq = res ** 2
        C = np.sum(res_sq)
        print(f'{C = }\n')
        
        dC_dz2 = -2 * res
        dz2_da2 = 1
        dz2_db2 = 1
        da2_dw2 = self.z1
        da2_dz1 = self.w2
        dz1_da1 = softplus_der(self.a1)
        da1_dw1 = self.x
        da1_db1 = 1
        
        dC_db2 = dC_dz2 * dz2_db2
        dC_dw2 = dC_dz2 * dz2_da2 * da2_dw2
        dC_db1 = dC_dz2 * dz2_da2 * da2_dz1 * dz1_da1 * da1_db1
        dC_dw1 = dC_dz2 * dz2_da2 * da2_dz1 * dz1_da1 * da1_dw1

        self.b2 = self.b2 - np.sum(dC_db2) * self.step_size
        self.w2 = self.w2 - np.sum(dC_dw2, axis=1).reshape(2, 1) * self.step_size
        self.b1 = self.b1 - np.sum(dC_db1, axis=1).reshape(2, 1) * self.step_size
        self.w1 = self.w1 - np.sum(dC_dw1, axis=1).reshape(2, 1) * self.step_size

    def train(self, num_iter):
        for i in range(num_iter):
            self.backpropagate(i)

In [ ]:
net = FFNet()
net.train(10000)

x = np.arange(0, 1, 0.01)
plt.plot(x, net.calc(x))
plt.plot()